<a href="https://colab.research.google.com/github/NadiaSouza1304/Hack_DS_01/blob/bureau_balance/bureau_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PoD Academy - Analise de Credito para Fintech
A PoD Bank, Fintech que concede crédito para população com pouca informação de crédito, precisa de modelos (ML) para concessão de crédito

In [13]:
import pandas as pd
import numpy as np

###Instalando o pyspark

In [14]:
!pip install pyspark

###Mapeando o Google Drive, para possibilitar o acesso aos arquivos de leitura

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Etapas obrigatórias no pré-processamento de dados com Spark

Etapa 1: Importar a SparkSession para iniciar a sessão PySpark

In [16]:
from pyspark.sql import SparkSession

Etapa 2: Utilizar a função getOrCreate() para criar a sessão do Apache Spark

In [17]:
spark = SparkSession \
    .builder \
    .appName("Hackaton") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

Etapa 3: Verificar se o objeto spark contém em um formato graphical

In [18]:
spark

###Lendo os arquivos de formado csv

# 01
- O dataframe **df_colunas** contém a descrição das colunas dos outros dfs

In [19]:
df_colunas = spark.read.csv("/content/gdrive/MyDrive/11 - Hackaton/dados/HomeCredit_columns_description.csv", header=True)
df_colunas.createOrReplaceTempView("df_colunas")
df_colunas.show(truncate=False)

+---+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|_c0|Table                       |Row                       |Description                                                                                                                                                                                |Special                              |
+---+----------------------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------+
|1  |application_{train|test}.csv|SK_ID_CURR                |ID of loan in our sample                                                   

# 02
- O dataframe **df_application_train** contém os dados de treino

In [82]:
df_application_train = spark.read.csv('/content/gdrive/MyDrive/11 - Hackaton/dados/previous_application.csv', header=True)
df_application_train.createOrReplaceTempView("df_application_train")
df_application_train.show(10, truncate=False)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURP

In [81]:
df_application_train = spark.read.csv('/content/gdrive/MyDrive/11 - Hackaton/dados/application_train.csv', header=True)
df_application_train.createOrReplaceTempView("df_application_train")
df_application_train.show(10, truncate=False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+-----------------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+------------------+------------------+------------------+------------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------------+--------------------+-

# 06
- O dataframe **df_bureau** contém dados transacionais de saldos mensais dos créditos anteriores
 - Fornece dados de crédito de outras instituições financeiras.

In [21]:
df_bureau = spark.read.csv('/content/gdrive/MyDrive/11 - Hackaton/dados/bureau.csv', header=True)
df_bureau.createOrReplaceTempView("df_bureau")
df_bureau.show(10, truncate=False)

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0         

# 08
- O dataframe **df_bureau_balance** contém dados transacionais de saldos mensais dos créditos anteriores
  - Informações mensais sobre créditos anteriores do cliente em outras instituições financeiras.

In [22]:
df_bureau_balance = spark.read.csv('/content/gdrive/MyDrive/11 - Hackaton/dados/bureau_balance.csv', header=True)
df_bureau_balance.createOrReplaceTempView("df_bureau_balance")
df_bureau_balance.show(10, truncate=False)

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|5715448     |0             |C     |
|5715448     |-1            |C     |
|5715448     |-2            |C     |
|5715448     |-3            |C     |
|5715448     |-4            |C     |
|5715448     |-5            |C     |
|5715448     |-6            |C     |
|5715448     |-7            |C     |
|5715448     |-8            |C     |
|5715448     |-9            |0     |
+------------+--------------+------+
only showing top 10 rows



###Consulta spark.sql

- Lendo os dados da tabela application_train


In [23]:
spark.sql("""
SELECT
    *
FROM
    df_application_train
LIMIT 20
""").show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+------------------+------------------+------------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------------+--------------------+-------------

- Lendo os dados da tabela bureau_balance

In [27]:
spark.sql("""
SELECT
    *
FROM
    df_bureau_balance
LIMIT 20
""").show()

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



- Lendo os dados da tabela bureau

In [18]:
spark.sql("""
SELECT
    *
FROM
    df_bureau
LIMIT 20
""").show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

- Contando a quantidade de linhas das tabelas application_train e bureau

In [19]:
spark.sql("""
SELECT
    COUNT(SK_ID_CURR) AS Quantidade_SK_ID
FROM
    df_application_train
""").show()

+----------------+
|Quantidade_SK_ID|
+----------------+
|          215257|
+----------------+



In [20]:
spark.sql("""
SELECT
    COUNT (DISTINCT SK_ID_CURR) AS Quantidade_SK_ID_UNICOS
FROM
    df_application_train
""").show()

+-----------------------+
|Quantidade_SK_ID_UNICOS|
+-----------------------+
|                 215257|
+-----------------------+



In [21]:
spark.sql("""
SELECT
    Count(*) AS Quantidadede_Transacoes
FROM
    df_bureau
""").show()

+-----------------------+
|Quantidadede_Transacoes|
+-----------------------+
|                1716428|
+-----------------------+



In [22]:
# Quantidade de IDs únicos df_bureau_balance
spark.sql("""
SELECT
    COUNT(DISTINCT SK_ID_CURR) AS Quantidade_SK_ID_UNICOS
FROM
    df_bureau
""").show()

+-----------------------+
|Quantidade_SK_ID_UNICOS|
+-----------------------+
|                 305811|
+-----------------------+



In [23]:
# quantidade de transações iguais a Credit card
spark.sql("""
SELECT
    COUNT(DISTINCT SK_ID_CURR) AS TotalCreditCard
FROM
    df_bureau
WHERE
    CREDIT_TYPE = 'Credit card'
""").show()

+---------------+
|TotalCreditCard|
+---------------+
|         199965|
+---------------+



In [24]:
# quantidade de transações iguais a Credit card
spark.sql("""
SELECT
    COUNT(DISTINCT SK_ID_CURR) AS TotalRealEstateLoan
FROM
    df_bureau
WHERE
    CREDIT_TYPE = 'Real estate loan'
""").show()

+-------------------+
|TotalRealEstateLoan|
+-------------------+
|                 27|
+-------------------+



In [25]:
# quantidade de transações iguais a Credit card
spark.sql("""
SELECT
    COUNT(DISTINCT SK_ID_CURR) AS TotalCarLoan
FROM
    df_bureau
WHERE
    CREDIT_TYPE = 'Car loan'
""").show()

+------------+
|TotalCarLoan|
+------------+
|       22796|
+------------+



In [26]:
spark.sql("""
SELECT
    NAME_CONTRACT_TYPE,
    COUNT(*)
FROM
    df_application_train
GROUP BY
    NAME_CONTRACT_TYPE
ORDER BY
    COUNT(*)
""").show(truncate=False)

+------------------+--------+
|NAME_CONTRACT_TYPE|count(1)|
+------------------+--------+
|Revolving loans   |20594   |
|Cash loans        |194663  |
+------------------+--------+



In [27]:
spark.sql("""
SELECT
    CREDIT_TYPE,
    COUNT(*)
FROM
    df_bureau
GROUP BY
    CREDIT_TYPE
ORDER BY
    COUNT(*)
""").show(truncate=False)

+--------------------------------------------+--------+
|CREDIT_TYPE                                 |count(1)|
+--------------------------------------------+--------+
|Mobile operator loan                        |1       |
|Interbank credit                            |1       |
|Loan for purchase of shares (margin lending)|4       |
|Loan for the purchase of equipment          |19      |
|Real estate loan                            |27      |
|Cash loan (non-earmarked)                   |56      |
|Loan for working capital replenishment      |469     |
|Unknown type of loan                        |555     |
|Another type of loan                        |1017    |
|Loan for business development               |1975    |
|Microloan                                   |12413   |
|Mortgage                                    |18391   |
|Car loan                                    |27690   |
|Credit card                                 |402195  |
|Consumer credit                             |12

**### A PARTIR DAQUI Não faz parte do Hackaton é só pra ter uma ideia do que fazer.**

###Análise exploratória de uma chave SK_ID_CURR

Pelo modelo entidade relacionamento (MER) tem-se que as tabelas application_train e bureau se relacionam pela chave 'SK_ID_CURR'.

- Lendo os dados do SK_ID_CURR na tabela applicantion_train

In [50]:
id_curr = '100002'

In [51]:
spark.sql(f"""
SELECT
    *
FROM
    df_application_train
WHERE
    SK_ID_CURR = {id_curr}
""").show()


+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------+------------+------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+----

- Lendo os dados do SK_ID_CURR = '100002' na tabela bureau

In [52]:
spark.sql(f"""
SELECT
    *
FROM
    df_bureau
WHERE
    SK_ID_CURR = {id_curr}
""").show()


+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    100002|     6158904|       Closed|     currency 1|      -1125|                 0|            -1038.0|          -1038

Através da vizualização da chave SK_ID_CURR = '100002' vemos que para um cadastro (tabela application_train) pode-se ter várias transações (tabela bureau).


- Analisando os dados categóricos da tabela bureau, colunas:
  - CREDIT_ACTIVE
  - CREDIT_CURRENCY
  - CREDIT_TYPE

In [ ]:
spark.sql(f"""
SELECT
    SK_ID_CURR,
    CREDIT_ACTIVE,
    CREDIT_CURRENCY,
    CREDIT_TYPE
FROM
    df_bureau
WHERE
    SK_ID_CURR = {id_curr}
""").show()

+----------+-------------+---------------+---------------+
|SK_ID_CURR|CREDIT_ACTIVE|CREDIT_CURRENCY|    CREDIT_TYPE|
+----------+-------------+---------------+---------------+
|    100002|       Closed|     currency 1|    Credit card|
|    100002|       Closed|     currency 1|    Credit card|
|    100002|       Closed|     currency 1|Consumer credit|
|    100002|       Closed|     currency 1|Consumer credit|
|    100002|       Closed|     currency 1|Consumer credit|
|    100002|       Active|     currency 1|    Credit card|
|    100002|       Active|     currency 1|Consumer credit|
|    100002|       Closed|     currency 1|    Credit card|
+----------+-------------+---------------+---------------+



Analisando a tabela acima vemos que cada coluna categórica possui domínios variados.

###Metodologia

- Como o foco principal é enriquecer a tabela de treino application_train com mais colunas, de modo que explique melhor as transações que determinado indivíduo fez, temos que agrupar as categorias em colunas.

- Para isso vamos criar colunas agregadas como:
  - SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END)
  
- A rotina acima cria uma coluna agregada que quando o crédito for igual a 'Closed' então conta como 1 senão é zero, e como está dentro de um SUM, cada vez que achar um 'Closed' vai somar 1. Com isso eu crio uma variável que pode ter vários créditos em situação 'Closed'.  Após isso, posso pegar esse dado e juntar com a tabela application_train, pois agora eu só tenho 1 linha para um único ID e não mais várias linhas.

- Com isso, podemos aplicar o máximo de conceitos de negócio, de combinação de conceitos e criar variáveis massivamente, ou seja, podemos criar um books de variáveis, com foco de aumentar o poder preditivo na parte de modelagem.


###Identificando todos domínios das variáveis categóricas da tabela 'bureau':
  - CREDIT_ACTIVE
  - CREDIT_CURRENCY
  - CREDIT_TYPE

In [53]:
spark.sql("""
SELECT
    CREDIT_ACTIVE,
    COUNT(*)
FROM
    df_bureau
GROUP BY
    CREDIT_ACTIVE
ORDER BY
    COUNT(*)
""").show()

+-------------+--------+
|CREDIT_ACTIVE|count(1)|
+-------------+--------+
|     Bad debt|      21|
|         Sold|    6527|
|       Active|  630607|
|       Closed| 1079273|
+-------------+--------+



In [54]:
spark.sql("""
SELECT
    CREDIT_CURRENCY,
    COUNT(*)
FROM
    df_bureau
GROUP BY
    CREDIT_CURRENCY
ORDER BY
    COUNT(*)
""").show()

+---------------+--------+
|CREDIT_CURRENCY|count(1)|
+---------------+--------+
|     currency 4|      10|
|     currency 3|     174|
|     currency 2|    1224|
|     currency 1| 1715020|
+---------------+--------+



In [55]:
spark.sql("""
SELECT
    CREDIT_TYPE,
    COUNT(*)
FROM
    df_bureau
GROUP BY
    CREDIT_TYPE
ORDER BY
    COUNT(*)
""").show(truncate=False)

+--------------------------------------------+--------+
|CREDIT_TYPE                                 |count(1)|
+--------------------------------------------+--------+
|Mobile operator loan                        |1       |
|Interbank credit                            |1       |
|Loan for purchase of shares (margin lending)|4       |
|Loan for the purchase of equipment          |19      |
|Real estate loan                            |27      |
|Cash loan (non-earmarked)                   |56      |
|Loan for working capital replenishment      |469     |
|Unknown type of loan                        |555     |
|Another type of loan                        |1017    |
|Loan for business development               |1975    |
|Microloan                                   |12413   |
|Mortgage                                    |18391   |
|Car loan                                    |27690   |
|Credit card                                 |402195  |
|Consumer credit                             |12

In [82]:
spark.sql("""
SELECT
    COUNT(*)
FROM
    df_bureau
""").show()

+--------+
|count(1)|
+--------+
| 1716428|
+--------+



###Transformando as variáveis categóricas em variáveis numéricas

- Agregando apenas as variáveis categóricas e seus domínios

In [30]:
spark.sql("""
SELECT
    *
FROM
    df_bureau_balance
""").show()

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



In [57]:
df_bureau_cat = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END) AS qtd_credit_active_closed,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END) AS qtd_credit_active_active,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) AS qtd_credit_active_bad_debt,
  SUM(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END) AS qtd_credit_active_sold,

  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN 1 ELSE 0 END) AS qtd_credit_currency_4,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN 1 ELSE 0 END) AS qtd_credit_currency_3,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN 1 ELSE 0 END) AS qtd_credit_currency_2,
  SUM(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN 1 ELSE 0 END) AS qtd_credit_currency_1,

  SUM(CASE WHEN CREDIT_Type = 'Mobile operator loan' THEN 1 ELSE 0 END) AS qtd_credit_type_mobile_operator_loan,
  SUM(CASE WHEN CREDIT_Type = 'Interbank credit' THEN 1 ELSE 0 END) AS qtd_credit_type_interbank_credit,
  SUM(CASE WHEN CREDIT_Type = 'Loan for purchase of shares (margin lending)' THEN 1 ELSE 0 END) AS qtd_credit_type_loan_for_purchase_of_shares,
  SUM(CASE WHEN CREDIT_Type = 'Loan for the purchase of equipment' THEN 1 ELSE 0 END) AS qtd_credit_type_mobile_operator_loan_for_the_purchase_of_equipament,
  SUM(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_Type = 'Cash loan (non-earmarked)' THEN 1 ELSE 0 END) AS qtd_credit_type_cash_loan,
  SUM(CASE WHEN CREDIT_Type = 'Loan for working capital replenishment' THEN 1 ELSE 0 END) AS qtd_credit_type_loan_for_working_capital_replenishment,
  SUM(CASE WHEN CREDIT_Type = 'Unknown type of loan' THEN 1 ELSE 0 END) AS qtd_credit_type_unknown_type_of_loan,
  SUM(CASE WHEN CREDIT_Type = 'Another type of loan' THEN 1 ELSE 0 END) AS qtd_credit_type_another_type_of_loan,
  SUM(CASE WHEN CREDIT_Type = 'Loan for business development' THEN 1 ELSE 0 END) AS qtd_credit_type_loan_for_business_development,
  SUM(CASE WHEN CREDIT_Type = 'Microloan' THEN 1 ELSE 0 END) AS qtd_credit_type_microloan,
  SUM(CASE WHEN CREDIT_Type = 'Mortgage' THEN 1 ELSE 0 END) AS qtd_credit_type_mortgage,
  SUM(CASE WHEN CREDIT_Type = 'Car loan ' THEN 1 ELSE 0 END) AS qtd_credit_type_car_loan,
  SUM(CASE WHEN CREDIT_Type = 'Credit card' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan,
  SUM(CASE WHEN CREDIT_Type = 'Consumer credit' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan,

  MIN(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END) AS min_credit_active_closed,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END) AS min_credit_active_active,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) AS min_credit_active_bad_debt,
  MIN(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END) AS min_credit_active_sold,

  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN 1 ELSE 0 END) AS min_credit_currency_4,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN 1 ELSE 0 END) AS min_credit_currency_3,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN 1 ELSE 0 END) AS min_credit_currency_2,
  MIN(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN 1 ELSE 0 END) AS min_credit_currency_1,

  MIN(CASE WHEN CREDIT_Type = 'Mobile operator loan' THEN 1 ELSE 0 END) AS min_credit_type_mobile_operator_loan,
  MIN(CASE WHEN CREDIT_Type = 'Interbank credit' THEN 1 ELSE 0 END) AS min_credit_type_interbank_credit,
  MIN(CASE WHEN CREDIT_Type = 'Loan for purchase of shares (margin lending)' THEN 1 ELSE 0 END) AS min_credit_type_loan_for_purchase_of_shares,
  MIN(CASE WHEN CREDIT_Type = 'Loan for the purchase of equipment' THEN 1 ELSE 0 END) AS min_credit_type_mobile_operator_loan_for_the_purchase_of_equipament,
  MIN(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS min_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_Type = 'Cash loan (non-earmarked)' THEN 1 ELSE 0 END) AS min_credit_type_cash_loan,
  MIN(CASE WHEN CREDIT_Type = 'Loan for working capital replenishment' THEN 1 ELSE 0 END) AS min_credit_type_loan_for_working_capital_replenishment,
  MIN(CASE WHEN CREDIT_Type = 'Unknown type of loan' THEN 1 ELSE 0 END) AS min_credit_type_unknown_type_of_loan,
  MIN(CASE WHEN CREDIT_Type = 'Another type of loan' THEN 1 ELSE 0 END) AS min_credit_type_another_type_of_loan,
  MIN(CASE WHEN CREDIT_Type = 'Loan for business development' THEN 1 ELSE 0 END) AS min_credit_type_loan_for_business_development,
  MIN(CASE WHEN CREDIT_Type = 'Microloan' THEN 1 ELSE 0 END) AS min_credit_type_microloan,
  MIN(CASE WHEN CREDIT_Type = 'Mortgage' THEN 1 ELSE 0 END) AS min_credit_type_mortgage,
  MIN(CASE WHEN CREDIT_Type = 'Car loan ' THEN 1 ELSE 0 END) AS min_credit_type_car_loan,
  MIN(CASE WHEN CREDIT_Type = 'Credit card' THEN 1 ELSE 0 END) AS min_credit_type_real_estate_loan,
  MIN(CASE WHEN CREDIT_Type = 'Consumer credit' THEN 1 ELSE 0 END) AS min_credit_type_real_estate_loan,

  MAX(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END) AS max_credit_active_closed,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END) AS max_credit_active_active,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) AS max_credit_active_bad_debt,
  MAX(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END) AS max_credit_active_sold,

  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN 1 ELSE 0 END) AS max_credit_currency_4,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN 1 ELSE 0 END) AS max_credit_currency_3,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN 1 ELSE 0 END) AS max_credit_currency_2,
  MAX(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN 1 ELSE 0 END) AS max_credit_currency_1,

  MAX(CASE WHEN CREDIT_Type = 'Mobile operator loan' THEN 1 ELSE 0 END) AS max_credit_type_mobile_operator_loan,
  MAX(CASE WHEN CREDIT_Type = 'Interbank credit' THEN 1 ELSE 0 END) AS max_credit_type_interbank_credit,
  MAX(CASE WHEN CREDIT_Type = 'Loan for purchase of shares (margin lending)' THEN 1 ELSE 0 END) AS max_credit_type_loan_for_purchase_of_shares,
  MAX(CASE WHEN CREDIT_Type = 'Loan for the purchase of equipment' THEN 1 ELSE 0 END) AS max_credit_type_mobile_operator_loan_for_the_purchase_of_equipament,
  MAX(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS max_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_Type = 'Cash loan (non-earmarked)' THEN 1 ELSE 0 END) AS max_credit_type_cash_loan,
  MAX(CASE WHEN CREDIT_Type = 'Loan for working capital replenishment' THEN 1 ELSE 0 END) AS max_credit_type_loan_for_working_capital_replenishment,
  MAX(CASE WHEN CREDIT_Type = 'Unknown type of loan' THEN 1 ELSE 0 END) AS max_credit_type_unknown_type_of_loan,
  MAX(CASE WHEN CREDIT_Type = 'Another type of loan' THEN 1 ELSE 0 END) AS max_credit_type_another_type_of_loan,
  MAX(CASE WHEN CREDIT_Type = 'Loan for business development' THEN 1 ELSE 0 END) AS max_credit_type_loan_for_business_development,
  MAX(CASE WHEN CREDIT_Type = 'Microloan' THEN 1 ELSE 0 END) AS max_credit_type_microloan,
  MAX(CASE WHEN CREDIT_Type = 'Mortgage' THEN 1 ELSE 0 END) AS max_credit_type_mortgage,
  MAX(CASE WHEN CREDIT_Type = 'Car loan ' THEN 1 ELSE 0 END) AS max_credit_type_car_loan,
  MAX(CASE WHEN CREDIT_Type = 'Credit card' THEN 1 ELSE 0 END) AS max_credit_type_real_estate_loan,
  MAX(CASE WHEN CREDIT_Type = 'Consumer credit' THEN 1 ELSE 0 END) AS max_credit_type_real_estate_loan,

  AVG(CASE WHEN CREDIT_ACTIVE = 'Closed' THEN 1 ELSE 0 END) AS avg_credit_active_closed,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Active' THEN 1 ELSE 0 END) AS avg_credit_active_active,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Bad debt' THEN 1 ELSE 0 END) AS avg_credit_active_bad_debt,
  AVG(CASE WHEN CREDIT_ACTIVE = 'Sold' THEN 1 ELSE 0 END) AS avg_credit_active_sold,

  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 4' THEN 1 ELSE 0 END) AS avg_credit_currency_4,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 3' THEN 1 ELSE 0 END) AS avg_credit_currency_3,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 2' THEN 1 ELSE 0 END) AS avg_credit_currency_2,
  AVG(CASE WHEN CREDIT_CURRENCY = 'currency 1' THEN 1 ELSE 0 END) AS avg_credit_currency_1,

  AVG(CASE WHEN CREDIT_Type = 'Mobile operator loan' THEN 1 ELSE 0 END) AS avg_credit_type_mobile_operator_loan,
  AVG(CASE WHEN CREDIT_Type = 'Interbank credit' THEN 1 ELSE 0 END) AS avg_credit_type_interbank_credit,
  AVG(CASE WHEN CREDIT_Type = 'Loan for purchase of shares (margin lending)' THEN 1 ELSE 0 END) AS avg_credit_type_loan_for_purchase_of_shares,
  AVG(CASE WHEN CREDIT_Type = 'Loan for the purchase of equipment' THEN 1 ELSE 0 END) AS avg_credit_type_mobile_operator_loan_for_the_purchase_of_equipament,
  AVG(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS avg_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_Type = 'Cash loan (non-earmarked)' THEN 1 ELSE 0 END) AS avg_credit_type_cash_loan,
  AVG(CASE WHEN CREDIT_Type = 'Loan for working capital replenishment' THEN 1 ELSE 0 END) AS avg_credit_type_loan_for_working_capital_replenishment,
  AVG(CASE WHEN CREDIT_Type = 'Unknown type of loan' THEN 1 ELSE 0 END) AS avg_credit_type_unknown_type_of_loan,
  AVG(CASE WHEN CREDIT_Type = 'Another type of loan' THEN 1 ELSE 0 END) AS avg_credit_type_another_type_of_loan,
  AVG(CASE WHEN CREDIT_Type = 'Loan for business development' THEN 1 ELSE 0 END) AS avg_credit_type_loan_for_business_development,
  AVG(CASE WHEN CREDIT_Type = 'Microloan' THEN 1 ELSE 0 END) AS avg_credit_type_microloan,
  AVG(CASE WHEN CREDIT_Type = 'Mortgage' THEN 1 ELSE 0 END) AS avg_credit_type_mortgage,
  AVG(CASE WHEN CREDIT_Type = 'Car loan ' THEN 1 ELSE 0 END) AS avg_credit_type_car_loan,
  AVG(CASE WHEN CREDIT_Type = 'Credit card' THEN 1 ELSE 0 END) AS avg_credit_type_real_estate_loan,
  AVG(CASE WHEN CREDIT_Type = 'Consumer credit' THEN 1 ELSE 0 END) AS avg_credit_type_real_estate_loan
FROM
  df_bureau
GROUP BY
  SK_ID_CURR
""")

Após criar o dataframe df_bureau_cat somente com agregações de variáveis categóricas, podemos criar outros dataframes com agregações entre as variáveis categóricas e numéricas. As variáveis numéricas da tabela bureau são:

- AMT_CREDIT_MAX_OVERDUE
- CNT_CREDIT_PROLONG
- AMT_CREDIT_SUM
- AMT_CREDIT_SUM_DEBT
- AMT_CREDIT_SUM_LIMIT
- AMT_CREDIT_SUM_OVERDUE
- AMT_ANNUITY

OBS: Para automatizar a criação de cada linha de agregação vamos desenvolver uma rotina que gera o texto de agregação. Após, basta susbstiruir na requesição SQL.

###Agregando cada variável numérica a cada variável categórica e seus domínios


1- Considerando a variável categórica CREDIT_ACTIVE


In [46]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CREDIT_ACTIVE']

# Criando os domínios

dominios_cat = ['Active', 'Closed', 'Bad debt', 'Sold']

# Criando variável numérica

var_num = ['AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', \
           'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', \
           'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY']

In [59]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

    #SUM(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan

SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_MAX_OVERDUE,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_CNT_CREDIT_PROLONG,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_DEBT,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_LIMIT,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_OVERDUE,
SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_ANNUITY,
SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_MAX_OVERDUE,
SUM(CASE WHEN CR

- Criando o dataframe df_bureau_credit_active_num

In [60]:
df_bureau_credit_active_num = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_ACTIVE_Active_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_ACTIVE_Closed_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_ACTIVE_Bad_debt_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_ACTIVE_Sold_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_ACTIVE_Active_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_ACTIVE_Closed_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_ACTIVE_Bad_debt_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_ACTIVE_Sold_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_ACTIVE_Active_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_ACTIVE_Closed_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_ACTIVE_Bad_debt_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_ACTIVE_Sold_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Active' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_ACTIVE_Active_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Closed' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_ACTIVE_Closed_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Bad debt' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_ACTIVE_Bad_debt_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_ACTIVE= 'Sold' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_ACTIVE_Sold_AMT_ANNUITY
FROM
  df_bureau
GROUP BY
  SK_ID_CURR
""")

2- Considerando a variável categórica CREDIT_CURRENCY

In [61]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CREDIT_CURRENCY']

# Criando os domínios

dominios_cat = ['currency 4', 'currency 3', 'currency 2', 'currency 1']

# Criando a variável numérica

var_num = ['AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', \
           'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', \
           'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY']

In [62]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')


SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_MAX_OVERDUE,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_CNT_CREDIT_PROLONG,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_DEBT,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_LIMIT,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_OVERDUE,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_ANNUITY,
SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_MA

- Criando o dataframe df_bureau_credit_currency_num

In [63]:
df_bureau_credit_currency_num = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_4_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_3_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_2_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_CURRENCY_currency_1_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_4_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_3_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_2_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_CURRENCY_currency_1_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_4_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_3_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_2_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_CURRENCY_currency_1_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 4' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_4_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 3' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_3_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 2' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_2_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_CURRENCY= 'currency 1' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_CURRENCY_currency_1_AMT_ANNUIT
FROM
  df_bureau
GROUP BY
  SK_ID_CURR
""")

3- Considerando a variável categórica CREDIT_TYPE

In [59]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['CREDIT_TYPE']

# Criando os domínios

dominios_cat = ['Mobile operator loan', 'Interbank credit', \
                'Loan for purchase of shares', \
                'Loan for the purchase of equipment', \
                'Real estate loan', 'Cash loan', \
                'Loan for working capital replenishment', \
                'Unknown type of loan', 'Another type of loan', \
                'Loan for business development', 'Microloan', \
                'Mortgage', 'Car loan', 'Credit card', 'Consumer credit']

# Criando a variável numérica

var_num = ['AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', \
           'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', \
           'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY']

In [65]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

    #SUM(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan

SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_MAX_OVERDUE,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_CNT_CREDIT_PROLONG,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_DEBT,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_LIMIT,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_OVERDUE,
SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_opera

- Craindo o dataframe df_bureau_credit_type_num

In [66]:
df_bureau_credit_type_num = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Mobile_operator_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Interbank_credit_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Real_estate_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Cash_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Unknown_type_of_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Another_type_of_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Loan_for_business_development_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Microloan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Mortgage_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Car_loan_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Credit_card_AMT_ANNUITY,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_MAX_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_CNT_CREDIT_PROLONG,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_DEBT,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_LIMIT,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_OVERDUE,
  SUM(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_ANNUITY ELSE 0 END) AS SUM_CREDIT_TYPE_Consumer_credit_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Mobile_operator_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Interbank_credit_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Real_estate_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Cash_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Unknown_type_of_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Another_type_of_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Loan_for_business_development_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Microloan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Mortgage_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Car_loan_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Credit_card_AMT_ANNUITY,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_MAX_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_CNT_CREDIT_PROLONG,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_DEBT,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_LIMIT,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_OVERDUE,
  MIN(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_ANNUITY ELSE 0 END) AS MIN_CREDIT_TYPE_Consumer_credit_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Mobile_operator_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Interbank_credit_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Real_estate_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Cash_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Unknown_type_of_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Another_type_of_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Loan_for_business_development_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Microloan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Mortgage_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Car_loan_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Credit_card_AMT_ANNUITY,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_MAX_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_CNT_CREDIT_PROLONG,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_DEBT,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_LIMIT,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_OVERDUE,
  MAX(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_ANNUITY ELSE 0 END) AS MAX_CREDIT_TYPE_Consumer_credit_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Mobile operator loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Mobile_operator_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Interbank credit' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Interbank_credit_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for purchase of shares' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_purchase_of_shares_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for the purchase of equipment' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_the_purchase_of_equipment_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Real estate loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Real_estate_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Cash loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Cash_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for working capital replenishment' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_working_capital_replenishment_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Unknown type of loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Unknown_type_of_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Another type of loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Another_type_of_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Loan for business development' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Loan_for_business_development_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Microloan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Microloan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Mortgage' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Mortgage_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Car loan' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Car_loan_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Credit card' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Credit_card_AMT_ANNUITY,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_MAX_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_MAX_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN CNT_CREDIT_PROLONG ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_CNT_CREDIT_PROLONG,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_DEBT ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_DEBT,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_LIMIT ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_LIMIT,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_CREDIT_SUM_OVERDUE ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_CREDIT_SUM_OVERDUE,
  AVG(CASE WHEN CREDIT_TYPE= 'Consumer credit' THEN AMT_ANNUITY ELSE 0 END) AS AVG_CREDIT_TYPE_Consumer_credit_AMT_ANNUITY
FROM
  df_bureau
GROUP BY
  SK_ID_CURR
""")

###Criando variáveis agregadas somente com as variáveis numéricas e datas da tabela bureau

As colunas numéricas e de datas são:

- AMT_CREDIT_MAX_OVERDUE
- CNT_CREDIT_PROLONG
- AMT_CREDIT_SUM
- AMT_CREDIT_SUM_DEBT
- AMT_CREDIT_SUM_LIMIT
- AMT_CREDIT_SUM_OVERDUE
- AMT_ANNUITY
- DAYS_CREDIT
- CREDIT_DAY_OVERDUE
- DAYS_CREDIT_ENDDATE
- DAYS_ENDDATE_FACT
- DAYS_CREDIT_UPDATE

In [67]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável numérica e de data

var_num = ['AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', \
           'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', \
           'AMT_CREDIT_SUM_OVERDUE', 'AMT_ANNUITY', 'DAYS_CREDIT',
           'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', \
           'DAYS_CREDIT_UPDATE']

In [68]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for var_n in var_num:
        print(ag + "(" + var_n + ") AS " + ag + "_" + var_n + ',')

    #SUM(AMT_CREDIT_MAX_OVERDUE) AS sum_credit_max_overdue

SUM(AMT_CREDIT_MAX_OVERDUE) AS SUM_AMT_CREDIT_MAX_OVERDUE,
SUM(CNT_CREDIT_PROLONG) AS SUM_CNT_CREDIT_PROLONG,
SUM(AMT_CREDIT_SUM) AS SUM_AMT_CREDIT_SUM,
SUM(AMT_CREDIT_SUM_DEBT) AS SUM_AMT_CREDIT_SUM_DEBT,
SUM(AMT_CREDIT_SUM_LIMIT) AS SUM_AMT_CREDIT_SUM_LIMIT,
SUM(AMT_CREDIT_SUM_OVERDUE) AS SUM_AMT_CREDIT_SUM_OVERDUE,
SUM(AMT_ANNUITY) AS SUM_AMT_ANNUITY,
SUM(DAYS_CREDIT) AS SUM_DAYS_CREDIT,
SUM(CREDIT_DAY_OVERDUE) AS SUM_CREDIT_DAY_OVERDUE,
SUM(DAYS_CREDIT_ENDDATE) AS SUM_DAYS_CREDIT_ENDDATE,
SUM(DAYS_ENDDATE_FACT) AS SUM_DAYS_ENDDATE_FACT,
SUM(DAYS_CREDIT_UPDATE) AS SUM_DAYS_CREDIT_UPDATE,
MIN(AMT_CREDIT_MAX_OVERDUE) AS MIN_AMT_CREDIT_MAX_OVERDUE,
MIN(CNT_CREDIT_PROLONG) AS MIN_CNT_CREDIT_PROLONG,
MIN(AMT_CREDIT_SUM) AS MIN_AMT_CREDIT_SUM,
MIN(AMT_CREDIT_SUM_DEBT) AS MIN_AMT_CREDIT_SUM_DEBT,
MIN(AMT_CREDIT_SUM_LIMIT) AS MIN_AMT_CREDIT_SUM_LIMIT,
MIN(AMT_CREDIT_SUM_OVERDUE) AS MIN_AMT_CREDIT_SUM_OVERDUE,
MIN(AMT_ANNUITY) AS MIN_AMT_ANNUITY,
MIN(DAYS_CREDIT) AS MIN_DAYS_CREDIT,
MIN(CRED

- Criando o dataframe df_num_bureau

In [69]:
df_num_bureau = spark.sql("""
SELECT
  SK_ID_CURR,
  SUM(AMT_CREDIT_MAX_OVERDUE) AS SUM_AMT_CREDIT_MAX_OVERDUE,
  SUM(CNT_CREDIT_PROLONG) AS SUM_CNT_CREDIT_PROLONG,
  SUM(AMT_CREDIT_SUM) AS SUM_AMT_CREDIT_SUM,
  SUM(AMT_CREDIT_SUM_DEBT) AS SUM_AMT_CREDIT_SUM_DEBT,
  SUM(AMT_CREDIT_SUM_LIMIT) AS SUM_AMT_CREDIT_SUM_LIMIT,
  SUM(AMT_CREDIT_SUM_OVERDUE) AS SUM_AMT_CREDIT_SUM_OVERDUE,
  SUM(AMT_ANNUITY) AS SUM_AMT_ANNUITY,
  SUM(DAYS_CREDIT) AS SUM_DAYS_CREDIT,
  SUM(CREDIT_DAY_OVERDUE) AS SUM_CREDIT_DAY_OVERDUE,
  SUM(DAYS_CREDIT_ENDDATE) AS SUM_DAYS_CREDIT_ENDDATE,
  SUM(DAYS_ENDDATE_FACT) AS SUM_DAYS_ENDDATE_FACT,
  SUM(DAYS_CREDIT_UPDATE) AS SUM_DAYS_CREDIT_UPDATE,
  MIN(AMT_CREDIT_MAX_OVERDUE) AS MIN_AMT_CREDIT_MAX_OVERDUE,
  MIN(CNT_CREDIT_PROLONG) AS MIN_CNT_CREDIT_PROLONG,
  MIN(AMT_CREDIT_SUM) AS MIN_AMT_CREDIT_SUM,
  MIN(AMT_CREDIT_SUM_DEBT) AS MIN_AMT_CREDIT_SUM_DEBT,
  MIN(AMT_CREDIT_SUM_LIMIT) AS MIN_AMT_CREDIT_SUM_LIMIT,
  MIN(AMT_CREDIT_SUM_OVERDUE) AS MIN_AMT_CREDIT_SUM_OVERDUE,
  MIN(AMT_ANNUITY) AS MIN_AMT_ANNUITY,
  MIN(DAYS_CREDIT) AS MIN_DAYS_CREDIT,
  MIN(CREDIT_DAY_OVERDUE) AS MIN_CREDIT_DAY_OVERDUE,
  MIN(DAYS_CREDIT_ENDDATE) AS MIN_DAYS_CREDIT_ENDDATE,
  MIN(DAYS_ENDDATE_FACT) AS MIN_DAYS_ENDDATE_FACT,
  MIN(DAYS_CREDIT_UPDATE) AS MIN_DAYS_CREDIT_UPDATE,
  MAX(AMT_CREDIT_MAX_OVERDUE) AS MAX_AMT_CREDIT_MAX_OVERDUE,
  MAX(CNT_CREDIT_PROLONG) AS MAX_CNT_CREDIT_PROLONG,
  MAX(AMT_CREDIT_SUM) AS MAX_AMT_CREDIT_SUM,
  MAX(AMT_CREDIT_SUM_DEBT) AS MAX_AMT_CREDIT_SUM_DEBT,
  MAX(AMT_CREDIT_SUM_LIMIT) AS MAX_AMT_CREDIT_SUM_LIMIT,
  MAX(AMT_CREDIT_SUM_OVERDUE) AS MAX_AMT_CREDIT_SUM_OVERDUE,
  MAX(AMT_ANNUITY) AS MAX_AMT_ANNUITY,
  MAX(DAYS_CREDIT) AS MAX_DAYS_CREDIT,
  MAX(CREDIT_DAY_OVERDUE) AS MAX_CREDIT_DAY_OVERDUE,
  MAX(DAYS_CREDIT_ENDDATE) AS MAX_DAYS_CREDIT_ENDDATE,
  MAX(DAYS_ENDDATE_FACT) AS MAX_DAYS_ENDDATE_FACT,
  MAX(DAYS_CREDIT_UPDATE) AS MAX_DAYS_CREDIT_UPDATE,
  AVG(AMT_CREDIT_MAX_OVERDUE) AS AVG_AMT_CREDIT_MAX_OVERDUE,
  AVG(CNT_CREDIT_PROLONG) AS AVG_CNT_CREDIT_PROLONG,
  AVG(AMT_CREDIT_SUM) AS AVG_AMT_CREDIT_SUM,
  AVG(AMT_CREDIT_SUM_DEBT) AS AVG_AMT_CREDIT_SUM_DEBT,
  AVG(AMT_CREDIT_SUM_LIMIT) AS AVG_AMT_CREDIT_SUM_LIMIT,
  AVG(AMT_CREDIT_SUM_OVERDUE) AS AVG_AMT_CREDIT_SUM_OVERDUE,
  AVG(AMT_ANNUITY) AS AVG_AMT_ANNUITY,
  AVG(DAYS_CREDIT) AS AVG_DAYS_CREDIT,
  AVG(CREDIT_DAY_OVERDUE) AS AVG_CREDIT_DAY_OVERDUE,
  AVG(DAYS_CREDIT_ENDDATE) AS AVG_DAYS_CREDIT_ENDDATE,
  AVG(DAYS_ENDDATE_FACT) AS AVG_DAYS_ENDDATE_FACT,
  AVG(DAYS_CREDIT_UPDATE) AS AVG_DAYS_CREDIT_UPDATE
FROM
  df_bureau
GROUP BY
  SK_ID_CURR
""")

###Criando variáveis numéricas que envolvem regras de negócio da tabela bureau

- Duração do Crédito, Crédito/Conta e Diferença de data
 - CREDIT_DURATION = DAYS_CREDIT + DAYS_CREDIT_ENDDATE
 - ENDDATE_DIF = DAYS_CREDIT_ENDDATE - DAYS_ENDDATE_FACT

- Taxa e Diferença de Credito para Débito:
 - DEBT_PERCENTAGE = AMT_CREDIT_SUM / AMT_CREDIT_SUM_DEBT
 - DEBT_CREDIT_DIFF = AMT_CREDIT_SUM - AMT_CREDIT_SUM_DEBT
 - CREDIT_TO_ANNUITY_RATIO = AMT_CREDIT_SUM / AMT_ANNUITY
 - BUREAU_CREDIT_FACT_DIFF = DAYS_CREDIT - DAYS_ENDDATE_FACT
 - BUREAU_CREDIT_ENDDATE_DIFF = DAYS_CREDIT - DAYS_CREDIT_ENDDATE
 - BUREAU_CREDIT_DEBT_RATIO = AMT_CREDIT_SUM_DEBT / AMT_CREDIT_SUM



- Criando o dataframe df_bureau_regras_data

In [70]:
df_bureau_regras_data = spark.sql("""
SELECT
  SK_ID_CURR,
  (DAYS_CREDIT + DAYS_CREDIT_ENDDATE) AS CREDIT_DURATION,
  (DAYS_CREDIT_ENDDATE - DAYS_ENDDATE_FACT) AS ENDDATE_DIF,
  ROUND((AMT_CREDIT_SUM / AMT_CREDIT_SUM_DEBT), 2) AS DEBT_PERCENTAGE,
  ROUND((AMT_CREDIT_SUM - AMT_CREDIT_SUM_DEBT), 2) AS DEBT_CREDIT_DIFF,
  ROUND((AMT_CREDIT_SUM / AMT_ANNUITY), 2) AS CREDIT_TO_ANNUITY_RATIO,
  ROUND((DAYS_CREDIT - DAYS_CREDIT_ENDDATE), 2) AS BUREAU_CREDIT_ENDDATE_DIFF,
  ROUND((AMT_CREDIT_SUM_DEBT /AMT_CREDIT_SUM), 2) AS BUREAU_CREDIT_DEBT_RATIO,
  (DAYS_CREDIT - DAYS_ENDDATE_FACT) AS BUREAU_CREDIT_FACT_DIFF
FROM
  df_bureau
""")


###Criando variáveis numéricas que envolvem regras de negócio da tabela application_train

- Idade:
 - AGE_YEARS = -DAYS_BIRTH / 365

- Porcentagens:
 - DAYS_EMPLOYED_PERC = DAYS_EMPLOYED / DAYS_BIRTH
 - INCOME_CREDIT_PERC = AMT_INCOME_TOTAL / AMT_CREDIT
 - INCOME_PER_PERSON = AMT_INCOME_TOTAL / CNT_FAM_MEMBERS
 - ANNUITY_INCOME_PERC = AMT_ANNUITY / AMT_INCOME_TOTAL
 - PAYMENT_RATE = AMT_ANNUITY / AMT_CREDIT

- Índice de Crédito
 - CREDIT_TO_GOODS_RATIO = AMT_CREDIT / AMT_GOODS_PRICE
    
- Índices de Receita
 - INCOME_TO_EMPLOYED_RATIO = AMT_INCOME_TOTAL / DAYS_EMPLOYED
 - INCOME_TO_BIRTH_RATIO = AMT_INCOME_TOTAL / DAYS_BIRTH
    
- Razões de tempo
 - ID_TO_BIRTH_RATIO = DAYS_ID_PUBLISH / DAYS_BIRTH
 - CAR_TO_BIRTH_RATIO = OWN_CAR_AGE / DAYS_BIRTH
 - CAR_TO_EMPLOYED_RATIO = OWN_CAR_AGE / DAYS_EMPLOYED
 - PHONE_TO_BIRTH_RATIO = DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH


- Criando o dataframe df_reg_application_train

In [72]:
df_reg_application_train = spark.sql("""
SELECT
    SK_ID_CURR,
    ROUND((-DAYS_BIRTH / 365), 1) AS AGE_YEARS,
    ROUND((DAYS_EMPLOYED / DAYS_BIRTH), 2) AS DAYS_EMPLOYED_PERC,
    ROUND((AMT_INCOME_TOTAL / AMT_CREDIT), 2) AS INCOME_CREDIT_PERC,
    ROUND((AMT_INCOME_TOTAL / CNT_FAM_MEMBERS), 2) AS INCOME_PER_PERSON,
    ROUND((AMT_ANNUITY / AMT_INCOME_TOTAL), 2) AS ANNUITY_INCOME_PERC,
    ROUND((AMT_ANNUITY / AMT_CREDIT), 2) AS PAYMENT_RATE,
    ROUND((AMT_CREDIT / AMT_GOODS_PRICE), 2) AS CREDIT_TO_GOODS_RATIO,
    ROUND((AMT_INCOME_TOTAL / DAYS_EMPLOYED), 2) AS INCOME_TO_EMPLOYED_RATIO,
    ROUND((AMT_INCOME_TOTAL / DAYS_BIRTH), 2) AS INCOME_TO_BIRTH_RATIO,
    ROUND((DAYS_ID_PUBLISH / DAYS_BIRTH), 2) AS ID_TO_BIRTH_RATIO,
    ROUND((OWN_CAR_AGE / DAYS_BIRTH), 2) AS CAR_TO_BIRTH_RATIO,
    ROUND((OWN_CAR_AGE / DAYS_EMPLOYED), 2) AS CAR_TO_EMPLOYED_RATIO,
    ROUND((DAYS_LAST_PHONE_CHANGE / DAYS_BIRTH), 2) AS PHONE_TO_BIRTH_RATIO
FROM
    df_application_train
""")

###Calculando o número de colunas de cada dataframe

Além do dataframe df_application_train que tem 122 colunas e do df_bureau que tem 17 colunas, criamos mais 7 novos dataframes utilizando agregações e regras de negócio, onde:

- df_bureau_cat: 93 colunas
- df_bureau_credit_active_num: 113 colunas
- df_bureau_credit_currency_num: 113 colunas
- df_bureau_credit_type_num: 421 colunas
- df_num_bureau: 49 colunas
- df_bureau_regras_data: 9 colunas
- df_reg_application_train: 14 colunas

Lembrando que para todos os dataframes temos que o número de variáveis é igual ao número de colunas - 1, pois a coluna 'SK_ID_CURR' é a chave de todos os dataframes. Então, inicialmente tínhamos 121 variáveis para a tabela df_application_train e 16 variáveis para o df_bureau, que ao todo são 137 variáveis.
Abaixo tem uma rotina que calcula no número de colunase os nomes da colunas.


In [ ]:
# Conta o número de colunas na tabela
num_colunas = len(df_reg_application_train.columns)

# Obtém os nomes das colunas
nomes_colunas = df_reg_application_train.columns

# Imprime o resultado
print("O df_reg_application_train possui ", num_colunas, "colunas.")
print('')
print('Os nomes das colunas:')
for nome_coluna in nomes_colunas:
    print(nome_coluna)

O df_reg_application_train possui  14 colunas.

Os nomes das colunas:
SK_ID_CURR
AGE_YEARS
DAYS_EMPLOYED_PERC
INCOME_CREDIT_PERC
INCOME_PER_PERSON
ANNUITY_INCOME_PERC
PAYMENT_RATE
CREDIT_TO_GOODS_RATIO
INCOME_TO_EMPLOYED_RATIO
INCOME_TO_BIRTH_RATIO
ID_TO_BIRTH_RATIO
CAR_TO_BIRTH_RATIO
CAR_TO_EMPLOYED_RATIO
PHONE_TO_BIRTH_RATIO


###Juntando ao df_application_train os 7 dataframes criados usando a coluna SK_ID_CURR

In [73]:
df_junto = df_application_train.join(df_bureau_cat, "SK_ID_CURR")\
               .join(df_bureau_credit_active_num, "SK_ID_CURR")\
               .join(df_bureau_credit_currency_num, "SK_ID_CURR")\
               .join(df_bureau_credit_type_num, "SK_ID_CURR")\
               .join(df_num_bureau, "SK_ID_CURR")\
               .join(df_bureau_regras_data, "SK_ID_CURR")\
               .join(df_reg_application_train, "SK_ID_CURR")

###Calculando o número de colunas do dataframe df_junto

In [74]:
# Contando o número de colunas do df_junto
num_colunas = len(df_junto.columns)

# Imprimindo o resultado
print(num_colunas, "colunas.")

977 colunas.


***Então, temos o dataframe final df_junto com 926 variáveis.
De 137 variáveis iniciais passamos para 815 variáveis, ou seja, aumentamos o número de variáveis em aproximandamente 6 vezes.***

###Conclusão

Por meio, do Pyspark SQL foi possível executar várias operações como filtragem, agregação e junção com dois conjuntos de dados de tamanho razoável, tendo 307511 linhas o df_application_train e 1716728 linhas o df_bureau, de forma eficiente.
também aumentamos o nosso book de variáveis com 815 novas variáveis, e com isso, aumentamos o poder preditvo na parte de modelagem em aproximadamente 6 vezes.

In [55]:
# Criando uma rotina que gera o texto da função de agregação

for i in range(3, 97, 3):
    print(f"CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - {i} AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_{i}_meses_flag,")

CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 3 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_3_meses_flag,
CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 6 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_6_meses_flag,
CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 9 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_9_meses_flag,
CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 12 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_12_meses_flag,
CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 15 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_15_meses_flag,
CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALA

In [71]:
spark.sql("""
SELECT
    *,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 3 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_3_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 6 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_6_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 9 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_9_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 12 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_12_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 15 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_15_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 18 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_18_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 21 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_21_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 24 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_24_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 27 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_27_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 30 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_30_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 33 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_33_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 36 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_36_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 39 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_39_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 42 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_42_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 45 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_45_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 48 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_48_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 51 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_51_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 54 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_54_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 57 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_57_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 60 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_60_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 63 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_63_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 66 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_66_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 69 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_69_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 72 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_72_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 75 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_75_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 78 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_78_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 81 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_81_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 84 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_84_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 87 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_87_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 90 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_90_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 93 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_93_meses_flag,
    CASE WHEN MONTHS_BALANCE BETWEEN (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) - 96 AND (SELECT MAX(MONTHS_BALANCE) FROM df_bureau_balance) THEN 1 ELSE 0 END AS ultimos_96_meses_flag
FROM
    df_bureau_balance
ORDER BY
    SK_ID_BUREAU, MONTHS_BALANCE DESC;
""").createOrReplaceTempView("df_temp")#.show()

In [61]:
spark.sql("""select distinct(STATUS) from df_bureau_balance""").show()

+------+
|STATUS|
+------+
|     3|
|     0|
|     5|
|     C|
|     X|
|     1|
|     4|
|     2|
+------+



In [76]:
# Criando as agregações
agg = ['SUM']

# Criando a variável categórica
cat = ['STATUS']

# Criando os domínios
dominios_cat = ['C', 'X', '0', '1', '2', '3', '4', '5']

# # Criando a variável numérica
# var_num = ['1']

# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for i in range(3, 97, 3):
        print(f"{ag}(CASE WHEN {cate} = '{dominio}' AND ultimos_{i}_meses_flag = 1 THEN 1 ELSE NULL END) AS {ag}_{cate}_{dominio}_U{i}M,")# + ag + "_" + cate + "_" + dominio.replace(" ", "_") + ',')

    #SUM(CASE WHEN CREDIT_Type = 'Real estate loan' THEN 1 ELSE 0 END) AS qtd_credit_type_real_estate_loan

SUM(CASE WHEN STATUS = 'C' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U3M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U6M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U9M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U12M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U15M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U18M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U21M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U24M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U27M,
SUM(CASE WHEN STATUS = 'C' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U3

In [78]:
spark.sql("""
select
    SK_ID_BUREAU,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U3M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U6M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U9M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U12M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U15M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U18M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U21M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U24M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U27M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U30M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U33M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U36M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U39M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U42M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U45M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U48M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U51M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U54M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U57M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U60M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U63M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U66M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U69M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U72M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U75M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U78M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U81M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U84M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U87M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U90M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U93M,
    SUM(CASE WHEN STATUS = 'C' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_C_U96M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U3M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U6M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U9M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U12M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U15M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U18M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U21M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U24M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U27M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U30M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U33M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U36M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U39M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U42M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U45M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U48M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U51M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U54M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U57M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U60M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U63M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U66M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U69M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U72M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U75M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U78M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U81M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U84M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U87M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U90M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U93M,
    SUM(CASE WHEN STATUS = 'X' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_X_U96M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U3M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U6M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U9M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U12M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U15M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U18M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U21M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U24M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U27M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U30M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U33M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U36M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U39M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U42M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U45M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U48M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U51M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U54M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U57M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U60M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U63M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U66M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U69M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U72M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U75M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U78M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U81M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U84M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U87M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U90M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U93M,
    SUM(CASE WHEN STATUS = '0' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_0_U96M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U3M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U6M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U9M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U12M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U15M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U18M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U21M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U24M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U27M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U30M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U33M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U36M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U39M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U42M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U45M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U48M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U51M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U54M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U57M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U60M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U63M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U66M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U69M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U72M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U75M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U78M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U81M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U84M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U87M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U90M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U93M,
    SUM(CASE WHEN STATUS = '1' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_1_U96M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U3M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U6M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U9M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U12M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U15M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U18M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U21M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U24M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U27M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U30M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U33M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U36M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U39M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U42M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U45M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U48M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U51M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U54M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U57M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U60M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U63M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U66M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U69M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U72M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U75M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U78M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U81M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U84M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U87M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U90M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U93M,
    SUM(CASE WHEN STATUS = '2' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_2_U96M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U3M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U6M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U9M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U12M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U15M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U18M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U21M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U24M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U27M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U30M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U33M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U36M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U39M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U42M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U45M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U48M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U51M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U54M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U57M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U60M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U63M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U66M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U69M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U72M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U75M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U78M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U81M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U84M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U87M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U90M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U93M,
    SUM(CASE WHEN STATUS = '3' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_3_U96M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U3M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U6M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U9M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U12M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U15M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U18M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U21M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U24M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U27M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U30M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U33M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U36M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U39M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U42M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U45M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U48M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U51M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U54M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U57M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U60M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U63M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U66M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U69M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U72M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U75M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U78M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U81M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U84M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U87M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U90M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U93M,
    SUM(CASE WHEN STATUS = '4' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_4_U96M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_3_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U3M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_6_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U6M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_9_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U9M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_12_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U12M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_15_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U15M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_18_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U18M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_21_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U21M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_24_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U24M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_27_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U27M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_30_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U30M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_33_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U33M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_36_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U36M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_39_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U39M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_42_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U42M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_45_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U45M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_48_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U48M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_51_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U51M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_54_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U54M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_57_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U57M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_60_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U60M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_63_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U63M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_66_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U66M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_69_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U69M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_72_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U72M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_75_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U75M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_78_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U78M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_81_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U81M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_84_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U84M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_87_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U87M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_90_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U90M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_93_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U93M,
    SUM(CASE WHEN STATUS = '5' AND ultimos_96_meses_flag = 1 THEN 1 ELSE NULL END) AS SUM_STATUS_5_U96M
from
    df_temp
group by
    SK_ID_BUREAU
order by
    SK_ID_BUREAU
""").createOrReplaceTempView("df_temp_02")

In [85]:
spark.sql("""select * from df_bureau""").show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

In [84]:
spark.sql("""select * from df_bureau_balance""").show()

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



In [86]:
spark.sql("""
select * from df_temp limit 20
""").show()

+------------+--------------+------+--------------------+--------------------+--------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|ultimos_3_meses_flag|ultimos_6_meses_flag|ultimos_9_meses_flag|ultimos_12_meses_flag|ultimos_15_meses_flag|ultimos_18_meses_flag|ultimos_21_meses_flag|ultimos_24_meses_flag|ultimos_27_meses_flag|ultimos_30_meses_flag|ultimos_3

In [80]:
spark.sql("""
select * from df_temp_02 limit 20
""").show()

+------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+--

In [58]:
spark.sql("""select * from df_bureau limit 20""").show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153